In [1]:
from rayoptics.environment import *
import numpy as np
import random
from copy import deepcopy
# from autograd import grad
from scipy.optimize import minimize


/Users/scottgilden/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
isdark = False

In [3]:
# create an afocal model
d_afocal = 7
d_focus = 0.2
wv = 400.5618
pupil_r = 2.5
M = 40
n = 1.69
L1_r = 2.5
L2_r = 0.1
L3_r = 1
# max_angle = 0.25/180*np.pi
max_angle = 1/180*np.pi
total_height = 10

f2 = d_afocal / (M + 1)
f1 = d_afocal - f2
f3 = total_height - d_afocal - d_focus

R1_L1 = (n-1) * f1
R2_L1 = 100000
R1_L2 = -100000
R2_L2 = 100000
R1_L3 = (n-1) * f3
R2_L3 = 100000

d_L1 = (np.abs(R1_L1) - np.sqrt(R1_L1**2 - L1_r**2))
d_L2 = 0.1
d_L3 = (np.abs(R1_L3) - np.sqrt(R1_L3**2 - L3_r**2))

f2

0.17073170731707318

In [4]:
number_of_colors = 3
hexadecimal_alphabets = '0123456789ABCDEF'
random.seed(2)
color = ["#" + ''.join([random.choice(hexadecimal_alphabets) for j in
range(6)]) for i in range(number_of_colors)]
wv = 400.5618

from visualize.rays import visualize_rays
from visualize.lens import visualize_lens
import plotly.graph_objects as go

def visualize(sm):
    # set visualization parameters
  radius = 8
  rays = 15
  x_offsets = list(np.linspace(-radius, radius, rays))
  y_offsets = list(np.linspace(-radius, radius, rays))
  z_offsets = [0] * rays
  num_rays = 1

  ray_data = visualize_rays(sm, radius=radius*0.9, wv=wv, x_offsets=x_offsets, y_offsets=y_offsets, num_rays=num_rays)
  lens_data = visualize_lens(sm, radius=radius)

  # plot
  data = []
  data.extend(ray_data)
  data.extend(lens_data)
  fig = go.Figure(data = data)
  fig.update_scenes(aspectmode='data')
  fig.show()


  return

ModuleNotFoundError: No module named 'visualize'

In [ ]:

def lens(f, R1, R2, d_L, pupil_r, p = [0], surf_idx = 0):
    opm = OpticalModel()
    sm  = opm['seq_model']
    osp = opm['optical_spec']
    pm = opm['parax_model']
    em = opm['ele_model']
    pt = opm['part_tree']
    ar = opm['analysis_results']
    wv = 400.5618
    osp['pupil'] = PupilSpec(osp, key=['object', 'pupil'], value=pupil_r*2)
    osp['fov'] = FieldSpec(osp, key=['object', 'angle'], value=2.5, flds=[0., 0.707, 1.], is_relative=True)
    osp['wvls'] = WvlSpec([('F', 0.5), (wv, 1.0), ('C', 0.5)], ref_wl=1)

    opm.radius_mode = True

    sm.gaps[0].thi = 10 #starting distance to first lens
    remaining_height = np.abs(f)


    sm.add_surface([R1, 1], p, f, surf_idx)
    #sm.set_stop()
    sm.add_surface([R1, 0])
    
    """
    if p != 0:
      sm.ifcs[1].phase_element = doe.DiffractiveElement(coefficients=p,phase_fct = doe.radial_phase_fct)

    """
    #sm.gaps[-1].thi = remaining_height - d_L
    sm.gaps[-1].thi = 100 #distance from last lens
    opm.update_model()


    return sm

In [ ]:
f = 0
R1 = 100000
R2 = 100000
d_L = d_L1
pupil = L1_r
max_angle = 1/180*np.pi
sm = lens(f, R1, R2, d_L, pupil) #no diffraction, flat lens
visualize(sm)

In [ ]:
f = 50
sm = lens(f, R1, R2, d_L, pupil, 0,1) #w/ diffraction
visualize(sm)
